## 1. import necessary libraries
In the module "local navigation" of this project, we will need to use several external libraries to complete the tache.

In [ ]:
import sys
import time
import cv2
import serial
import math
from threading import Timer
import numpy as np
sys.path.append("localNavigation")
from Thymio import Thymio

## 2. Define constants
We define some constants which will be used for several usages: 
- to scale the motors command in the desired interval
- to correctly define the PID controler
- to avoid the uncertainty of robot's behavior.


In [2]:
# used for defining the position tolerance while moving forward to a coordinate
SENSOR_SCALE = 1500
MEMORY_FACTOR = 10
BASE_SPEED_HIGH = 150
BASE_SPEED_LOW = 75

# PID controler
KP = 100
KI = 3.5
KD = 8
ERROR_SATURATION = 10

# tolerance for unprecision
TOLERENCE_POSITION = 10

## 3. Define global variables
We define equally several global variables which will be useful for local navigation. These variables will be used by different motor commande functions for several times. The variable group "error" is useful for computing PID controler, and the memorised speeds will be useful in case of ANN local avoidance.  

In [ ]:
value_proximity = [0,0,0,0,0,0,0]  # stores horizontal proximity measurements 
value_speed = [0,0]                # stores motors' speeds measurements 
actual_position = [0,0]            # stores actual position  
actual_angle = 0                   # stores actual angle with x axis 
actual_goal = [0,0]                # stores next goal position 
no_detection = False               # stores boolien flag for detection of obstacle.  

# These variables will be used for PID controller 
error_sum = 0                      
error = 0
error_prev=0

# These variables will be used for local avoidance (memory)
speed_avoidance_l_prev = 0
speed_avoidance_r_prev = 0

##  4. Definition of fondamental functions:
- Timer
- Thymio connexion
- sensor measurements
- accessor of measurement variables
- updoad actual position 
There are several functions which should be called in a regular interval of time, such as the filter and the Thymio sampling measurements, because the calculation of them need the intergration. The function "RepeatedTimer" allows to call an arbitary function in a custormized interval. 



In [ ]:
class RepeatedTimer(object):
    """
    definition of timer, which can be called in a fixed interval
    """
    def __init__(self, interval, function, *args, **kwargs):
        self._timer     = None
        self.interval   = interval
        self.function   = function
        self.args       = args
        self.kwargs     = kwargs
        self.is_running = False
        self.start()

    def _run(self):
        self.is_running = False
        self.start()
        self.function(*self.args, **self.kwargs)

    def start(self):
        if not self.is_running:
            self._timer = Timer(self.interval, self._run)
            self._timer.start()
            self.is_running = True

    def stop(self):
        self._timer.cancel()
        self.is_running = False


In [ ]:
def connexion_thymio():
    """
    This function should be called in order to connect Thymio
    "Thymio is connected :)" will be sent once the connexion is successful
    """
    global th
    th = Thymio.serial(port="COM5", refreshing_rate=0.1)
    time.sleep(10) # To make sure the Thymio has had time to connect
    print("Thymio is connected :)")

In [ ]:
def deconnexion_thymio():
    """
    Deconnect Thymio once mission is accomplished
    """
    time.sleep(1)
    th.close()


In [ ]:
def measure_sensor():
    """
    read the measurements of Thymio. The measurements 
    datas will be stored in global variables
    """
    global value_proximity
    global value_speed

    value_proximity=th['prox.horizontal']
    value_speed=[th['motor.left.speed'],th['motor.right.speed']]
    for i in range(2):
        if value_speed[i]>600:
            value_speed[i]=value_speed[i]-2**16
    return value_proximity,value_speed

In [ ]:
def get_position(robot_pos):
    """
    This function gets the result of Kalman filter, a variable containing the information of position 
    and angle of this robot, unpack this variable to extract and to store 
    these information into the global variables
    
    param: 
    robot_pos :containing position of robot, angle of robot, bool whether robot is detected
    
    return: 
    actual_position,actual_angle : actual position and angle of robot
    """
    
    global actual_angle
    global actual_position
    global no_detection
  
    if robot_pos[2]==True:        # if the robot is detected
        no_detection= False
        actual_position= [robot_pos[0][0],robot_pos[0][1]] # upload actual state of robot
        actual_angle=robot_pos[1]
        return actual_position,actual_angle
    else:                                                  # if robot is not detected
        no_detection=True
        return actual_position,actual_angle                # keep the previous state of robot 

## 5. Motors control functions

In [ ]:
def move(l_speed=500, r_speed=500, verbose=False):
    """
    Sets the motor speeds of the Thymio 
    param l_speed: left motor speed
    param r_speed: right motor speed
    param verbose: whether to print status messages or not
    """
    # Printing the speeds if requested
    if verbose: print("\t\t Setting speed : ", l_speed, r_speed)
    
    # Changing negative values to the expected ones with the bitwise complement
    l_speed = l_speed if l_speed>=0 else 2**16+l_speed
    r_speed = r_speed if r_speed>=0 else 2**16+r_speed

    # Setting the motor speeds
    th.set_var("motor.left.target", l_speed)
    th.set_var("motor.right.target", r_speed)

In [ ]:
def stop(verbose=False):
    """
    param verbose: whether to print status messages or not
    """
    # Printing the speeds if requested
    if verbose:
        print("\t\t Stopping")

    # Setting the motor speeds
    th.set_var("motor.left.target", 0)
    th.set_var("motor.right.target", 0)

## 6. Local navigation and motor control :
In our project, the trajectory of robot is constitued by a series of coordinates of passage points. In order to make Thymio  move to a given coordinate (here goal) and be able to avoid the obstacles near to it on the way, we control the Thymio with <b>3 speeds in parallel </b> : 
- <b>Basis speed</b> on the both right and left wheels for moving forward. 
- <b>Differential speeds</b> on 2 wheels for aligning to the goal direction.
- <b>Differential speeds</b> on 2 wheels for avoiding the obstacles.


### Trajectory tracking with PID controler

In order to track correctly the trajectory, our strategy consists of making aligne Thymio all the time to the goal while moving forward. Here we will define the <b>"error angle"</b> as the how much Thymio should rotate in order to aligne with the goal. During the tracking, Thymio computes in each iteration the error angle and commands the motors in fonction of this error. The error angle is computed based on the knowlege of Thymio's actual position, goal position and Thymio's actual orientation. A simplified schemas can illustrate this geometry problem, we will adapt the sign convention of angle, positive for clockwise and negative for counter clockwise: 

<div>
<img src="error_compute.PNG" width="700"/>
</div>

Once the error angle is obtained, Thymio will move. The motors command of Thymio at time T is generated by a PID controler, which controls the wheels speed in using the error. We simply compute the $\sum_{t=0}^T e(t)$ by summing up the errors from beginning $e(0)$ to the moment T $e(T)$, and $\delta error$ by calculating the difference between actual error $e(T)$ and $e(T-1)$, the error at time T-1.

$$CommandInSpeed(T)= K_P * e(T) + K_I * \sum_{t=0}^T e(t) + K_D * \delta e(t) $$

Then comes to experiment and tune the PID parameters: $K_P$,$K_D$ and $K_I$.
Our robot is expected to rotate quickly to aligne to the goal direction while moving forward, so we need high $K_P$ to increase <b>the reaction of robot due to error</b>, then $K_I$ is added to <b>vanish the permenant angle error</b>. However, we observe that Thymio moved with lots of overshoot, so the term $K_D$ is added at the end to <b>reduce this overshoot</b>. We also put an <b>anti-windup system </b> which saturates the sum of error, in order to avoid not only saturation of actuators, but also the excess violent action. 




### Local avoidance using  Artificial Neural Networks with Memory



The objective of local avoidance is that Thymio should detect and avoid the obstacles on its way, while keeping tracking the trajectory to the goal as good as possible. To achieve this behavior, the speed command of Thymio should not only contain the avoidance speed, but also the PID control speed which intent to align Thymio's orientation to the goal. 

The major difficulty in the design of a such avoidance speed controler comes from the fact that, Thymio has only proximity sensors in front and behind, but not on the sides. Since the local avoidance imposes that the obstacle should only be detected by proximity sensors, camera should not give any information about it to Thymio. Once Thymio successes to avoid the obstacle, the moment that the obstacle is not visible to Thymio (at time T=t+2h in the figure), Thymio will be controled only by PID tracking speed. Since the term of error and the sum of error are strongly increased during the avoidance phase, the PID controler will give a very high command on motors in order to aligne Thymio to the goal, which leads to the collision. 
To minimize this accident, we implement the <b>avoidance controler</b> in giving Thymio the capacity to <i>"memorize"</i>. This controler is based on the <b>artificial neural networks</b>.

<div>
<img src="memory_comparaison.PNG" width="700"/>
</div>

The working principe of our ANN controler is below: for each iteration, 7 proximity sensors measure the environment, these measurements are stored in an array x[0:7]. Each sensor measurment is assigned to a weight, the weights of all 7 sensors measurements are stored in the arrays w_l[0:7] and w_r[0:7], the amplitude of weight means the importance of the measurement to the wheels' speed, the signe of weight impact the direction of wheels' speed.
Concerning the memory part, x[7] and x[8] store the "degraded memory". The final speeds of left and right wheels are obtained by the multiplication between x array and w array. With this approch, the actual avoidance speed is not only dependent on proiomity sensors measurements, but also on the <i>"memory"</i> of previous avoidance speeds, so that Thymio can stay in avoidance phase longer, in order to better pass through the obstacle without collision (see Figure right).



In [1]:
def calculate_error(actual_position,goal,actual_angle):
    """
    This function computes the error between the actual 
    angle and the target angle, we will try to eliminate
    this error with PID controler 
    
    params: 
    actual_position,goal,actual_angle
    
    return: 
    error
    """
    
    global error_sum 
    global no_detection
    global error
    global error_prev
    
    goal_array = np.array([goal[0],goal[1]])
    actual_position_array = np.array([actual_position[0],actual_position[1]])
   
    direction = goal_array - actual_position_array
    angle = np.arctan2(direction[1], direction[0])
    error_prev = error
    error = -actual_angle + angle 

    if error < -np.pi:
        error += 2*np.pi 
    if error >  np.pi:
        error -= 2*np.pi 

    error_sum += error

    return error

In [ ]:
def follow_the_way_to_dream(actual_position,goal,actual_angle):
    '''
    This function aims to command the motors of Thymio based on
    the angle error previously calculated.
    
    It has 2 parts of controls to the motor: 
        - speed for tracking the way to goal if no obstacle is detected
        - speed of local avoidance if the obstacle is detected
    
    params: 
    actual_position,goal,actual_angle
    '''
    global error_sum 
    global value_proximity
    global speed_avoidance_l_prev
    global speed_avoidance_r_prev

    

    if no_detection == False:

        '''
        part 1: local avoidance motor speed control
        Thymio will turn in order to avoid the obstacle
        base on the proximity sensor, it uses the memory 
        in order to "memorize" the existance of obstacle. 
        So Thymio can turn more to avoid it.
        
        '''
        x = np.array([0,0,0,0,0,0,0,0,0])      # array containing the measurement datas and memorized speeds
        
        
        # ponderation of importance of each sensor contributing the rotation [1:7] 
        # amplitude of movement for each motor due to avoidance [8:9]
        w_l = np.array([40,  20, -20, -20, -40,  30, -10, 8, 0])
        w_r = np.array([-40, -20, 20,  20,  40, -10,  30, 0, 8])

        x[:7]= np.array(value_proximity) / SENSOR_SCALE     # compute the roration due to obstacle
        x[7] = speed_avoidance_l_prev / MEMORY_FACTOR       # memory degradation
        x[8] = speed_avoidance_r_prev / MEMORY_FACTOR 

        speed_avoidance_l = np.sum(x * w_l)
        speed_avoidance_r = np.sum(x * w_r)
        speed_avoidance_l_prev=speed_avoidance_l
        speed_avoidance_r_prev=speed_avoidance_r


        #if memory to abstacle avoidance mode is still existing, higher basis speed to pass through
        if x[7] != 0 or x[8] != 0:
            base_speed = BASE_SPEED_HIGH
        else : 
            base_speed = BASE_SPEED_LOW

        '''
        part 2: PID motor speed control
        Thymio will try to ajuste its move orientation and turn in order to aligne 
        to the goal all the time
        
        '''

        error = calculate_error(actual_position,goal,actual_angle)
        

        if error_sum > ERROR_SATURATION:
            error_sum = ERROR_SATURATION
        if error_sum < -ERROR_SATURATION:
            error_sum = -ERROR_SATURATION
        
        
        # Compute the speed relative to PID controler
        vitesse_PID = KP * error + KI * error_sum + KD *(error-error_prev)

        # combining the final speed 
        speed_l = base_speed + vitesse_PID + speed_avoidance_l
        speed_r = base_speed - vitesse_PID + speed_avoidance_r

        move(int(speed_l),int(speed_r))
  
    else:
        stop()

## 7. Defining the next destination
Since there will be a several points generated in the list of trajectory, Thymio should pass them one after another, so it's inportant for Thymio to know which point is its actual destination. 
Firstly, in the initialization, we will assigne to Thymio the first point which is in the list of destination. Then, Thymio will move. 
The function detect_trajectoy aims to compare the actual position of Thymio and the actual goal to go. If the position of Thymio is in side of the cercle centered in actual goal with the radius TOLERENCE_POSITION, then we consider that Thymio arrive at the goal, so we update the actual goal to go. 

In [ ]:
def detect_trajectory(actual_position,goal_actual): 
    """
    param: actual_position,goal_actual
    return: True or False depending on if Thymio is on the goal   
    """ 
    if (abs(goal_actual[0]-actual_position[0]) <= TOLERENCE_POSITION) and (abs(goal_actual[1]-actual_position[1])<=TOLERENCE_POSITION):   
        return True
    else:
        return False


In [1]:
def mission_accomplished():
    """
    param: none
    This function prints "Mission is accomplished!! " when Thymio has passed all the points  
    """ 
    stop()
    print("Mission is accomplished!! ")
